# RxAvans lecture 5

Last time you wrote the `log`, `tap`, `filter`, and `map` functions. These are pure, as they don't need to keep any state while processing each piece of data that goes through the pipeline. 

This time we'll write two functions that do require to keep some state: `take` and `skip`. But with lecture 5 you know you can keep state in functions by closures! 

Let's get to work!

First we look at how we can keep state between items of data the the producer produces. Below is an example of a sum link. It keeps a sum of all the values it has seen. The example is a bit silly, because we can't do anything with the sum. However, it does show that with a mutable variable that is enclosed by the function we return we can keep state between calls of the anonymous function.

In [ ]:
let sum previous =
    let mutable total = 0
    fun next -> 
        previous (fun data -> // this lambda encloses total
            total <- total + data // each item that is passed through the pipeline gets added to the total
            printfn "total: %i" total
            next data // we pass on the data to the next in the pipeline
        )

let producer observer = for i in 1..5 do observer i
let summed = sum producer

summed (printfn "received: %i")

We ready to get to work on `take`. The idea of this function is that it *takes* `n` items from the producer and ignores the rest. So in essence it is only listening to the first `n` items that go through the pipeline. For reference, RxJS also has a [take operator](https://rxjs.dev/api/operators/take). 

The overal structure of `take` is the same as `sum` in the example above, but we to need do something different with the mutable variable. Can you make it work?

In [8]:
let take amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data ->
            if count < amount then next data
            count <- count + 1
        )

// test your code with this snippet
let producer observer = for i in 1..10 do observer i
let taken = take 3 producer

taken (printfn "received: %i") // should print "received: 1", "received: 2", and "received: 3"

//write a function called 'passRepeated' that passes repeated values through the pipeline, so 1,2,2,2,3,3,4,2 should return 2,2,3





received: 1
received: 2
received: 3
received: 1
received: 2
received: 3
received: 4


Nice! Now let's write the `skip` function. `skip` will ignore the first `n` items before it starts passing on values through the pipeline. So the first `n` items are *skipped*.

Try to implement it!

In [ ]:
let skip amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data -> 
            if (count >= amount) then next data
            count <- count + 1
        )

// test your code with this snippet
let producer observer = for i in 1..10 do observer i
let skipped = skip 6 producer

skipped (printfn "received: %i") // should print "received: 7", "received: 8", "received: 9", "received: 10"

Here is an example with both `take` and `skip`. Run it to see if your output is correct and check that you understand what is happening!

In [ ]:
let producer observer = for i in 1..10 do observer i
let skipped = skip 5 producer
let taken = take 3 skipped

taken (printfn "received: %i") // should print "received: 6", "received: 7", "received: 8"

For completeness you find the full RxAvans code below. This will be the starting code of the final exam problem.

In [4]:
let log previous =
    fun next ->
        previous (fun data -> 
            printfn "logging: %i" data // this line was added
            next data
        )

let tap func previous = 
    fun next ->
        previous (fun data ->
            func data
            next data
        )

let filter predicate previous =
    fun next -> 
        previous (fun data -> 
            if predicate data then next data
        )

let map mapper previous =
    fun next ->
        previous (fun data ->
            next (mapper data)
        )

let take amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data ->
            if count < amount then next data
            count <- count + 1
        )

let skip amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data -> 
            if (count >= amount) then next data
            count <- count + 1
        )


Stopped due to error


Error: input.fsx (57,17)-(57,31) typecheck error The type '(int * int * int * int * int * int * int * int)' is not compatible with any of the types byte,int16,int32,int64,sbyte,uint16,uint32,uint64,nativeint,unativeint, arising from the use of a printf-style format string

This was the last RxAvans exercise sheet. Lecture 6 gives you a deeper understanding of functional programming concepts and a nice way to design your types, but for RxAvans you are ready for the exam! 🤞